# More SQL parsing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mo_sql_parsing as mosp

## SELECT

In [3]:
mosp.parse("SELECT id AS i FROM foo")

{'select': {'name': 'i', 'value': 'id'}, 'from': 'foo'}

In [4]:
mosp.parse("SELECT id AS i, key AS k FROM foo")

{'select': [{'name': 'i', 'value': 'id'}, {'name': 'k', 'value': 'key'}],
 'from': 'foo'}

In [5]:
mosp.parse("SELECT MIN(id) FROM foo")

{'select': {'value': {'min': 'id'}}, 'from': 'foo'}

In [6]:
mosp.parse("SELECT id + (SELECT MIN(id) FROM foo) FROM foo")

{'select': {'value': {'add': ['id',
    {'select': {'value': {'min': 'id'}}, 'from': 'foo'}]}},
 'from': 'foo'}

In [7]:
mosp.parse("SELECT DISTINCT id FROM foo")

{'select_distinct': {'value': 'id'}, 'from': 'foo'}

In [8]:
mosp.parse("SELECT MIN(id) AS m FROM foo")

{'select': {'name': 'm', 'value': {'min': 'id'}}, 'from': 'foo'}

In [9]:
mosp.parse("SELECT COUNT(DISTINCT id), MIN(id) FROM foo")

{'select': [{'value': {'distinct': True, 'count': 'id'}},
  {'value': {'min': 'id'}}],
 'from': 'foo'}

In [10]:
mosp.parse("SELECT NULL")

{'select': {'value': {'null': {}}}}

## Basic predicates

In [11]:
mosp.parse("SELECT * FROM foo, bar WHERE id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', 'bar'],
 'where': {'eq': ['id', 'fkey']}}

In [12]:
mosp.parse("SELECT * FROM foo, bar WHERE id != fkey")

{'select': {'all_columns': {}},
 'from': ['foo', 'bar'],
 'where': {'neq': ['id', 'fkey']}}

In [13]:
mosp.parse("SELECT * FROM foo WHERE id = 'bar'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'eq': ['id', {'literal': 'bar'}]}}

In [14]:
mosp.parse("SELECT * FROM foo WHERE id LIKE 'bar'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'like': ['id', {'literal': 'bar'}]}}

In [15]:
mosp.parse("SELECT * FROM foo WHERE id NOT LIKE 'bar'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'not_like': ['id', {'literal': 'bar'}]}}

In [16]:
mosp.parse("SELECT * FROM foo WHERE id < '1990-01-10'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'lt': ['id', {'literal': '1990-01-10'}]}}

In [17]:
mosp.parse("SELECT * FROM foo WHERE id = 42")

{'select': {'all_columns': {}}, 'from': 'foo', 'where': {'eq': ['id', 42]}}

## Compound predicates

In [18]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND key = 'bar'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]}, {'eq': ['key', {'literal': 'bar'}]}]}}

In [19]:
mosp.parse("SELECT * FROM foo WHERE id = 42 OR key = 'bar'")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'or': [{'eq': ['id', 42]}, {'eq': ['key', {'literal': 'bar'}]}]}}

In [20]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND (key = 'bar' OR key = 'baz')")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]},
   {'or': [{'eq': ['key', {'literal': 'bar'}]},
     {'eq': ['key', {'literal': 'baz'}]}]}]}}

In [21]:
mosp.parse("SELECT * FROM foo WHERE id = 42 AND NOT (key = 'bar' OR key = 'baz')")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'and': [{'eq': ['id', 42]},
   {'not': {'or': [{'eq': ['key', {'literal': 'bar'}]},
      {'eq': ['key', {'literal': 'baz'}]}]}}]}}

## Aliases

In [22]:
mosp.parse("SELECT * FROM foo f, bar b WHERE f.id = b.fkey")

{'select': {'all_columns': {}},
 'from': [{'value': 'foo', 'name': 'f'}, {'value': 'bar', 'name': 'b'}],
 'where': {'eq': ['f.id', 'b.fkey']}}

## Subqueries

In [23]:
mosp.parse("SELECT * FROM foo WHERE id IN (SELECT fkey FROM bar)")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'in': ['id', {'select': {'value': 'fkey'}, 'from': 'bar'}]}}

In [24]:
mosp.parse("SELECT * FROM foo, (SELECT * FROM bar WHERE bar_id < 42) WHERE foo_id != bar_id")

{'select': {'all_columns': {}},
 'from': ['foo',
  {'select': {'all_columns': {}},
   'from': 'bar',
   'where': {'lt': ['bar_id', 42]}}],
 'where': {'neq': ['foo_id', 'bar_id']}}

In [25]:
mosp.parse("SELECT * FROM foo, (SELECT * FROM bar WHERE bar_id < 42) AS b WHERE foo_id != bar_id")

{'select': {'all_columns': {}},
 'from': ['foo',
  {'value': {'select': {'all_columns': {}},
    'from': 'bar',
    'where': {'lt': ['bar_id', 42]}},
   'name': 'b'}],
 'where': {'neq': ['foo_id', 'bar_id']}}

## Special predicates

In [26]:
mosp.parse("SELECT * FROM foo, bar WHERE id IN ('bar', 'baz')")

{'select': {'all_columns': {}},
 'from': ['foo', 'bar'],
 'where': {'in': ['id', {'literal': ['bar', 'baz']}]}}

In [27]:
mosp.parse("SELECT * FROM foo WHERE id IN ('bar', (SELECT MIN(fkey) FROM bar))")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'in': ['id',
   [{'literal': 'bar'},
    {'select': {'value': {'min': 'fkey'}}, 'from': 'bar'}]]}}

In [28]:
mosp.parse("SELECT * FROM foo, bar WHERE id IN ('bar', bar.id, 'biz')")

{'select': {'all_columns': {}},
 'from': ['foo', 'bar'],
 'where': {'in': ['id', [{'literal': 'bar'}, 'bar.id', {'literal': 'biz'}]]}}

In [29]:
mosp.parse("SELECT * FROM foo WHERE id IN (SELECT id FROM bar)")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'in': ['id', {'select': {'value': 'id'}, 'from': 'bar'}]}}

In [30]:
mosp.parse("SELECT * FROM foo WHERE id BETWEEN 24 AND 42")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'between': ['id', 24, 42]}}

In [31]:
mosp.parse("SELECT * FROM foo WHERE id IS NOT NULL")

{'select': {'all_columns': {}}, 'from': 'foo', 'where': {'exists': 'id'}}

In [32]:
mosp.parse("SELECT * FROM foo WHERE id IS NULL")

{'select': {'all_columns': {}}, 'from': 'foo', 'where': {'missing': 'id'}}

## Advanced value manipulations

In [33]:
mosp.parse("SELECT * FROM foo WHERE id = CURRENT_DATE()")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'eq': ['id', {'current_date': {}}]}}

In [34]:
mosp.parse("SELECT * FROM foo WHERE id + '5 days'::interval = 42")

{'select': {'all_columns': {}},
 'from': 'foo',
 'where': {'eq': [{'add': ['id',
     {'cast': [{'literal': '5 days'}, {'interval': {}}]}]},
   42]}}

In [35]:
mosp.parse("SELECT id::interval FROM foo")

{'select': {'value': {'cast': ['id', {'interval': {}}]}}, 'from': 'foo'}

In [36]:
mosp.parse("SELECT CASE WHEN R.a = 42 THEN 'a' ELSE 'b' END FROM R")

{'select': {'value': {'case': [{'then': {'literal': 'a'},
     'when': {'eq': ['R.a', 42]}},
    {'literal': 'b'}]}},
 'from': 'R'}

In [37]:
mosp.parse("SELECT CASE WHEN R.a = 42 THEN 'a' WHEN R.a > 42 THEN 'b' ELSE 'c' END FROM R")

{'select': {'value': {'case': [{'then': {'literal': 'a'},
     'when': {'eq': ['R.a', 42]}},
    {'then': {'literal': 'b'}, 'when': {'gt': ['R.a', 42]}},
    {'literal': 'c'}]}},
 'from': 'R'}

In [38]:
mosp.parse("SELECT CASE WHEN R.a = 42 THEN 'a' END FROM R")

{'select': {'value': {'case': {'then': {'literal': 'a'},
    'when': {'eq': ['R.a', 42]}}}},
 'from': 'R'}

In [39]:
mosp.parse("SELECT 'prefix' || R.a FROM R")

{'select': {'value': {'concat': [{'literal': 'prefix'}, 'R.a']}}, 'from': 'R'}

In [40]:
mosp.parse("SELECT my_udf(R.a = S.b) FROM R, S")

{'select': {'value': {'my_udf': {'eq': ['R.a', 'S.b']}}}, 'from': ['R', 'S']}

## SELECT ... FROM ... JOIN ...

In [41]:
mosp.parse("SELECT * FROM foo JOIN bar")

{'select': {'all_columns': {}}, 'from': ['foo', {'join': 'bar'}]}

In [42]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [43]:
mosp.parse("SELECT * FROM foo f JOIN bar b")

{'select': {'all_columns': {}},
 'from': [{'value': 'foo', 'name': 'f'},
  {'join': {'value': 'bar', 'name': 'b'}}]}

In [44]:
mosp.parse("SELECT * FROM foo JOIN bar ON id = fkey AND id = 'baz'")

{'select': {'all_columns': {}},
 'from': ['foo',
  {'join': 'bar',
   'on': {'and': [{'eq': ['id', 'fkey']},
     {'eq': ['id', {'literal': 'baz'}]}]}}]}

In [45]:
mosp.parse("SELECT * FROM foo JOIN (SELECT * FROM bar) bar2 ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo',
  {'join': {'value': {'select': {'all_columns': {}}, 'from': 'bar'},
    'name': 'bar2'},
   'on': {'eq': ['id', 'fkey']}}]}

In [46]:
mosp.parse("SELECT * FROM foo CROSS JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'cross join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [47]:
mosp.parse("SELECT * FROM foo FULL JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'full join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [48]:
mosp.parse("SELECT * FROM foo LEFT JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'left join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [49]:
mosp.parse("SELECT * FROM foo OUTER JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'outer join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [50]:
mosp.parse("SELECT * FROM foo RIGHT OUTER JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'right outer join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [51]:
mosp.parse("SELECT * FROM foo INNER JOIN bar ON id = fkey")

{'select': {'all_columns': {}},
 'from': ['foo', {'inner join': 'bar', 'on': {'eq': ['id', 'fkey']}}]}

In [52]:
mosp.parse("SELECT * FROM foo NATURAL JOIN bar")

{'select': {'all_columns': {}}, 'from': ['foo', {'natural join': 'bar'}]}

In [53]:
mosp.parse("SELECT * FROM foo JOIN bar USING id")

{'select': {'all_columns': {}},
 'from': ['foo', {'join': 'bar', 'using': 'id'}]}

In [54]:
mosp.parse("SELECT * FROM foo JOIN bar USING (id, fkey)")

{'select': {'all_columns': {}},
 'from': ['foo', {'join': 'bar', 'using': ['id', 'fkey']}]}

In [55]:
mosp.parse("SELECT * FROM foo JOIN (bar JOIN baz ON a = b) ON c = d")

{'select': {'all_columns': {}},
 'from': ['foo',
  {'join': ['bar', {'join': 'baz', 'on': {'eq': ['a', 'b']}}],
   'on': {'eq': ['c', 'd']}}]}

## Advanced FROM clause / JOIN combinations

In [56]:
mosp.parse("SELECT * FROM R, S s JOIN R1 ON R.id = R1.id")

{'select': {'all_columns': {}},
 'from': ['R',
  {'value': 'S', 'name': 's'},
  {'join': 'R1', 'on': {'eq': ['R.id', 'R1.id']}}]}

In [57]:
mosp.parse("SELECT * FROM R, S s JOIN R1 r1 ON R.id = R1.id")

{'select': {'all_columns': {}},
 'from': ['R',
  {'value': 'S', 'name': 's'},
  {'join': {'value': 'R1', 'name': 'r1'}, 'on': {'eq': ['R.id', 'R1.id']}}]}

In [58]:
mosp.parse("SELECT * FROM R, S s JOIN (SELECT * FROM R1) ON R.id = R1.id")

{'select': {'all_columns': {}},
 'from': ['R',
  {'value': 'S', 'name': 's'},
  {'join': {'select': {'all_columns': {}}, 'from': 'R1'},
   'on': {'eq': ['R.id', 'R1.id']}}]}

In [59]:
mosp.parse("SELECT * FROM R, S s JOIN (SELECT * FROM R1) r1 ON R.id = R1.id")

{'select': {'all_columns': {}},
 'from': ['R',
  {'value': 'S', 'name': 's'},
  {'join': {'value': {'select': {'all_columns': {}}, 'from': 'R1'},
    'name': 'r1'},
   'on': {'eq': ['R.id', 'R1.id']}}]}

## EXPLAIN

In [60]:
mosp.parse("EXPLAIN SELECT * FROM foo, bar WHERE id = fkey")

{'explain': {'select': {'all_columns': {}},
  'from': ['foo', 'bar'],
  'where': {'eq': ['id', 'fkey']}}}

In [61]:
mosp.parse("EXPLAIN (ANALYZE, FORMAT JSON) SELECT * FROM foo, bar WHERE id = fkey")

{'analyze': True,
 'format': 'json',
 'explain': {'select': {'all_columns': {}},
  'from': ['foo', 'bar'],
  'where': {'eq': ['id', 'fkey']}}}

## Mathematics

In [62]:
mosp.parse("SELECT id + 5 FROM foo")

{'select': {'value': {'add': ['id', 5]}}, 'from': 'foo'}

In [63]:
mosp.parse("SELECT id - 5 FROM foo")

{'select': {'value': {'sub': ['id', 5]}}, 'from': 'foo'}

In [64]:
mosp.parse("SELECT id * 2 FROM foo")

{'select': {'value': {'mul': ['id', 2]}}, 'from': 'foo'}

In [65]:
mosp.parse("SELECT id / 1000 FROM foo")

{'select': {'value': {'div': ['id', 1000]}}, 'from': 'foo'}

In [66]:
mosp.parse("SELECT id % 2 FROM foo")

{'select': {'value': {'mod': ['id', 2]}}, 'from': 'foo'}

In [67]:
mosp.parse("SELECT -id FROM foo")

{'select': {'value': {'neg': 'id'}}, 'from': 'foo'}

## Aggregate Functions

In [68]:
mosp.parse("SELECT COUNT(*) FROM foo")

{'select': {'value': {'count': '*'}}, 'from': 'foo'}

In [69]:
mosp.parse("SELECT COUNT(id) FROM foo")

{'select': {'value': {'count': 'id'}}, 'from': 'foo'}

In [70]:
mosp.parse("SELECT AVG(id) FROM foo")

{'select': {'value': {'avg': 'id'}}, 'from': 'foo'}

In [71]:
mosp.parse("SELECT AVG('bar') FROM foo")

{'select': {'value': {'avg': {'literal': 'bar'}}}, 'from': 'foo'}

In [72]:
mosp.parse("SELECT MIN(id) FROM foo")

{'select': {'value': {'min': 'id'}}, 'from': 'foo'}

In [73]:
mosp.parse("SELECT MAX(id) FROM foo")

{'select': {'value': {'max': 'id'}}, 'from': 'foo'}

In [74]:
mosp.parse("SELECT SUM(id) FROM foo")

{'select': {'value': {'sum': 'id'}}, 'from': 'foo'}

## Function calls

In [75]:
mosp.parse("SELECT CURRENT_DATE() FROM foo WHERE TIMESTAMP() = id")

{'select': {'value': {'current_date': {}}},
 'from': 'foo',
 'where': {'eq': [{'timestamp': {}}, 'id']}}

In [76]:
mosp.parse("SELECT SOUNDEX(UPPER('id'))")

{'select': {'value': {'soundex': {'upper': {'literal': 'id'}}}}}

In [77]:
mosp.parse("SELECT REPEAT('id', 5)")

{'select': {'value': {'repeat': [{'literal': 'id'}, 5]}}}

In [78]:
mosp.parse("SELECT TIME(id) FROM foo")

{'select': {'value': {'time': 'id'}}, 'from': 'foo'}

In [79]:
mosp.parse("SELECT TIME(id OR key OR fkey) FROM foo")

{'select': {'value': {'time': {'or': ['id', 'key', 'fkey']}}}, 'from': 'foo'}

## GROUP BY, ORDER BY, HAVING, LIMIT

In [80]:
mosp.parse("SELECT * FROM foo GROUP BY id")

{'select': {'all_columns': {}}, 'from': 'foo', 'groupby': {'value': 'id'}}

In [81]:
mosp.parse("SELECT * FROM foo GROUP BY id, key")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': [{'value': 'id'}, {'value': 'key'}]}

In [82]:
mosp.parse("SELECT * FROM foo GROUP BY ALL id")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': {'value': 'ALL', 'name': 'id'}}

In [83]:
mosp.parse("SELECT * FROM foo GROUP BY DISTINCT id")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': {'value': {'distinct': 'id'}}}

In [84]:
mosp.parse("SELECT * FROM foo GROUP BY DISTINCT id, key")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': {'value': {'distinct': [{'value': 'id'}, {'value': 'key'}]}}}

In [85]:
mosp.parse("SELECT * FROM foo GROUP BY id HAVING SUM(id) = 42")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': {'value': 'id'},
 'having': {'eq': [{'sum': 'id'}, 42]}}

In [86]:
mosp.parse("SELECT * FROM foo GROUP BY id HAVING SUM(id) = 42 AND id = 1")

{'select': {'all_columns': {}},
 'from': 'foo',
 'groupby': {'value': 'id'},
 'having': {'and': [{'eq': [{'sum': 'id'}, 42]}, {'eq': ['id', 1]}]}}

In [87]:
mosp.parse("SELECT * FROM foo ORDER BY id")

{'select': {'all_columns': {}}, 'from': 'foo', 'orderby': {'value': 'id'}}

In [88]:
mosp.parse("SELECT * FROM foo ORDER BY id DESC")

{'select': {'all_columns': {}},
 'from': 'foo',
 'orderby': {'value': 'id', 'sort': 'desc'}}

In [89]:
mosp.parse("SELECT * FROM foo ORDER BY id, key")

{'select': {'all_columns': {}},
 'from': 'foo',
 'orderby': [{'value': 'id'}, {'value': 'key'}]}

In [90]:
mosp.parse("SELECT * FROM foo ORDER BY id DESC, key ASC")

{'select': {'all_columns': {}},
 'from': 'foo',
 'orderby': [{'value': 'id', 'sort': 'desc'}, {'value': 'key', 'sort': 'asc'}]}

In [91]:
mosp.parse("SELECT * FROM foo LIMIT 10 OFFSET 20")


{'select': {'all_columns': {}}, 'from': 'foo', 'limit': 10, 'offset': 20}

## Common Table Expressions


In [92]:
mosp.parse("""
WITH top100_posters AS (
	SELECT owner_user_id, COUNT(*) AS n_answers
	FROM answer, site
	WHERE site.site_name = 'stackoverflow'
		AND answer.site_id = site.site_id
	GROUP BY owner_user_id
	ORDER BY n_answers DESC
	LIMIT 100
)
SELECT account.display_name, top100_posters.n_answers
FROM account, so_user, site, top100_posters
WHERE site.site_name = 'stackoverflow'
	AND account.id = so_user.account_id
	AND site.site_id = so_user.site_id
	AND so_user.id = top100_posters.owner_user_id;
""")


{'select': [{'value': 'account.display_name'},
  {'value': 'top100_posters.n_answers'}],
 'from': ['account', 'so_user', 'site', 'top100_posters'],
 'where': {'and': [{'eq': ['site.site_name', {'literal': 'stackoverflow'}]},
   {'eq': ['account.id', 'so_user.account_id']},
   {'eq': ['site.site_id', 'so_user.site_id']},
   {'eq': ['so_user.id', 'top100_posters.owner_user_id']}]},
 'with': {'name': 'top100_posters',
  'value': {'select': [{'value': 'owner_user_id'},
    {'name': 'n_answers', 'value': {'count': '*'}}],
   'from': ['answer', 'site'],
   'where': {'and': [{'eq': ['site.site_name', {'literal': 'stackoverflow'}]},
     {'eq': ['answer.site_id', 'site.site_id']}]},
   'groupby': {'value': 'owner_user_id'},
   'orderby': {'value': 'n_answers', 'sort': 'desc'},
   'limit': 100}}}

In [93]:
mosp.parse("""
WITH t_a AS (SELECT 1),
    t_b AS (SELECT 2)
SELECT *
FROM R, t_a, t_b
""")

{'select': {'all_columns': {}},
 'from': ['R', 't_a', 't_b'],
 'with': [{'name': 't_a', 'value': {'select': {'value': 1}}},
  {'name': 't_b', 'value': {'select': {'value': 2}}}]}

## Window functions

In [94]:
mosp.parse("SELECT LAG(a) OVER () FROM R")

{'select': {'value': {'lag': 'a'}, 'over': {}}, 'from': 'R'}

In [95]:
mosp.parse("SELECT LAG(a) OVER () AS f FROM R")

{'select': {'name': 'f', 'value': {'lag': 'a'}, 'over': {}}, 'from': 'R'}

In [96]:
mosp.parse("SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname) FROM empsalary")

{'select': [{'value': 'depname'},
  {'value': 'empno'},
  {'value': 'salary'},
  {'value': {'avg': 'salary'}, 'over': {'partitionby': 'depname'}}],
 'from': 'empsalary'}

In [97]:
mosp.parse("""
SELECT depname, empno, salary,
    rank() OVER (PARTITION BY depname ORDER BY salary DESC)
FROM empsalary;
""")

{'select': [{'value': 'depname'},
  {'value': 'empno'},
  {'value': 'salary'},
  {'value': {'rank': {}},
   'over': {'partitionby': 'depname',
    'orderby': {'value': 'salary', 'sort': 'desc'}}}],
 'from': 'empsalary'}

In [98]:
mosp.parse("""
SELECT SUM(salary) OVER (PARTITION BY depname, empno ORDER BY salary DESC, empno)
FROM empsalary;
""")

{'select': {'value': {'sum': 'salary'},
  'over': {'partitionby': ['depname', 'empno'],
   'orderby': [{'value': 'salary', 'sort': 'desc'}, {'value': 'empno'}]}},
 'from': 'empsalary'}

In [99]:
# does (currently not work): filter is dropped
mosp.parse("""
SELECT SUM(salary) FILTER (WHERE salary > 100) OVER (PARTITION BY depname, empno ORDER BY salary DESC, empno)
FROM empsalary;
""")

{'select': {'value': {'sum': 'salary'}}, 'from': 'empsalary'}

## Parallel parsing

This does not work b/c MOSP maintains a global parser lock.
